In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import constants, units
from astropy.coordinates import Angle
from scipy.optimize import curve_fit
import pandas as pd
import os

In [2]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/MUSE/'

Load in coordinates and redshifts for my catalog

In [3]:
cat_file = np.loadtxt(rootdir + 'test_new_updated.cat')

In [4]:
gal_ids_arr = np.int_(cat_file[:,0])

In [5]:
x_world = cat_file[:,1]
y_world = cat_file[:,2]

In [6]:
x_world_deg = [x*units.deg for x in x_world]
y_world_deg = [y*units.deg for y in y_world]

In [7]:
x_world_rad = [x.to(units.rad) for x in x_world_deg]
y_world_rad = [y.to(units.rad) for y in y_world_deg]

In [8]:
ra_new_hms = [Angle(x).hms for x in x_world_deg]
dec_new_dms = [Angle(y).dms for y in y_world_deg]

In [9]:
ra_new_hms_str = ['%02d'%int(x[0])+':'+'%02d'%int(x[1])+':'+'%.2f'%np.round(x[2],2) for x in ra_new_hms]
dec_new_dms_str = ['+'+'%02d'%int(y[0])+':'+'%02d'%int(y[1])+':'+'%04.1f'%np.round(y[2],1) for y in dec_new_dms]

In [10]:
z_ids_arr = cat_file[:,12]

In [11]:
for i in range(len(ra_new_hms_str)):
    if(z_ids_arr[i]!=-1 and z_ids_arr[i]!=0):
        print('circle('+ra_new_hms_str[i]+','+dec_new_dms_str[i]+', 1.0") #color = cyan text = {'+'%.4f'%np.round(z_ids_arr[i], 4)+'}')

circle(15:24:25.75,+09:58:02.8, 1.0") #color = cyan text = {0.5356}
circle(15:24:25.21,+09:58:01.9, 1.0") #color = cyan text = {0.4581}
circle(15:24:23.05,+09:58:00.3, 1.0") #color = cyan text = {0.9576}
circle(15:24:24.95,+09:58:00.1, 1.0") #color = cyan text = {0.8217}
circle(15:24:23.61,+09:58:59.8, 1.0") #color = cyan text = {0.4784}
circle(15:24:23.36,+09:58:59.1, 1.0") #color = cyan text = {0.1393}
circle(15:24:22.99,+09:58:53.9, 1.0") #color = cyan text = {1.1935}
circle(15:24:23.92,+09:58:52.3, 1.0") #color = cyan text = {1.0959}
circle(15:24:23.52,+09:58:46.7, 1.0") #color = cyan text = {0.6469}
circle(15:24:26.25,+09:58:46.1, 1.0") #color = cyan text = {1.3379}
circle(15:24:25.48,+09:58:41.2, 1.0") #color = cyan text = {0.7010}
circle(15:24:24.80,+09:58:37.7, 1.0") #color = cyan text = {0.1204}
circle(15:24:24.69,+09:58:33.9, 1.0") #color = cyan text = {0.1203}
circle(15:24:25.66,+09:58:35.1, 1.0") #color = cyan text = {0.6531}
circle(15:24:23.54,+09:58:34.4, 1.0") #color = c

Put in custom redshifts

In [12]:
custom_coords_x = [154, 168, 276, 152, 111, 242, 212, 227]
custom_coords_y = [197, 174, 310, 242, 66, 69, 270, 105]
custom_redshifts = [1.3365, 1.3306, 0.5257, 0.1264, 0.6478, 0.1335, 1.1932, 0.8609]

Load in the MUSE FITS file

In [13]:
muse_fits = fits.open(rootdir + 'COMBINED_CUBE_MED_FINAL_vac_sub_15_pix.fits')

Extract the RA array

In [14]:
muse_fits.info()

Filename: /Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/MUSE/COMBINED_CUBE_MED_FINAL_vac_sub_15_pix.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU    1211   ()      
  1  DATA          1 ImageHDU        42   (337, 319, 3682)   float32   
  2  STAT          1 ImageHDU        43   (337, 319, 3682)   float32   


In [15]:
data_hdu = muse_fits[1]

In [16]:
data_hdu.header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    3 / number of data axes                            
NAXIS1  =                  337 / Axis lenght                                    
NAXIS2  =                  319 / Axis lenght                                    
NAXIS3  =                 3682 / Axis lenght                                    
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
EXTNAME = 'DATA    '           / This extension contains data values            
HDUCLASS= 'ESO     '           / class name (ESO format)                        
HDUDOC  = 'DICD    '           / document with class description                
HDUVERS = 'DICD version 6'     / version number (according to spec v2.5.1)      
HDUCLAS1= 'IMAGE   '        

Extract the MUSE cube as a 3-D array

In [17]:
spec_cube = data_hdu.data.copy()

In [18]:
spec_cube.shape

(3682, 319, 337)

Extract the RA array

In [19]:
RA_ref_pix = data_hdu.header['CRPIX1']-1
RA_ref_val = data_hdu.header['CRVAL1']
del_RA = data_hdu.header['CD1_1']

In [20]:
RA_0_val = RA_ref_val - del_RA*RA_ref_pix

In [21]:
RA_arr = np.arange(RA_0_val, RA_0_val + spec_cube.shape[2]*del_RA, del_RA)

In [22]:
RA_arr_deg = [Angle(ra, units.degree) for ra in RA_arr]

In [23]:
RA_arr_hms = [ra.hms for ra in RA_arr_deg]

In [24]:
RA_arr_labels = ['%02d'%int(x[0])+':'+'%02d'%int(x[1])+':'+'%.2f'%np.round(x[2],2) for x in RA_arr_hms]

Extract Dec array

In [25]:
Dec_ref_pix = data_hdu.header['CRPIX2']-1
Dec_ref_val = data_hdu.header['CRVAL2']
del_Dec = data_hdu.header['CD2_2']

In [26]:
Dec_0_val = Dec_ref_val - del_Dec*Dec_ref_pix

In [27]:
Dec_arr = np.arange(Dec_0_val, Dec_0_val + spec_cube.shape[1]*del_Dec, del_Dec)

In [28]:
Dec_arr_deg = [Angle(dec, units.degree) for dec in Dec_arr]

In [29]:
Dec_arr_dms = [dec.dms for dec in Dec_arr_deg]

In [30]:
Dec_arr_labels = ['+'+'%02d'%int(y[0])+':'+'%02d'%int(y[1])+':'+'%04.1f'%np.round(y[2],1) for y in Dec_arr_dms]

Put in RA/ Dec labels for custom objects

In [31]:
custom_coords_ra = []
custom_coords_dec = []

In [32]:
for i in range(len(custom_coords_x)):
    custom_coords_ra.append(RA_arr_labels[custom_coords_x[i]-1])
    custom_coords_dec.append(Dec_arr_labels[custom_coords_y[i]-1])

Print out rspec lines

In [33]:
for i in range(len(custom_coords_ra)):
        print('circle('+custom_coords_ra[i]+','+custom_coords_dec[i]+', 1.0") #color = red text = {'+'%.4f'%np.round(custom_redshifts[i], 4)+'}')

circle(15:24:24.70,+09:58:36.3, 1.0") #color = red text = {1.3365}
circle(15:24:24.51,+09:58:31.7, 1.0") #color = red text = {1.3306}
circle(15:24:23.07,+09:58:58.9, 1.0") #color = red text = {0.5257}
circle(15:24:24.73,+09:58:45.3, 1.0") #color = red text = {0.1264}
circle(15:24:25.27,+09:58:10.1, 1.0") #color = red text = {0.6478}
circle(15:24:23.53,+09:58:10.7, 1.0") #color = red text = {0.1335}
circle(15:24:23.93,+09:58:50.9, 1.0") #color = red text = {1.1932}
circle(15:24:23.73,+09:58:17.9, 1.0") #color = red text = {0.8609}
